# This model will combine the north and south shift in oder to see the difference

In [1]:
# dependencies copied from forward model
from okapy import rect_shear_fault
from math import sin, cos, tan, radians
from matplotlib import cm, colors
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#load in both data files
data_s = np.loadtxt('/home/karlee/earthquakes/alor_2015_JupNot/dsc_digitized/prepareInSARdata/south.okinv', delimiter=' ')  # make sure you have the correct path to the file for your system
data_s[:,0]*=1000  # convert x coord from km to m
data_s[:,1]*=1000  # convert y coord from km to m

data_n = np.loadtxt('/home/karlee/earthquakes/alor_2015_JupNot/dsc_digitized/prepareInSARdata/north.okinv', delimiter=' ')  # make sure you have the correct path to the file for your system
data_n[:,0]*=1000  # convert x coord from km to m
data_n[:,1]*=1000  # convert y coord from km to m

In [5]:
# a vector of fault parameters: strike, dip, rake, slip, x, y, length, top, bottom
#input the best fitting parameters from optimization
fparams = np.array([1.16198150e+02,  6.63744396e+01, -1.54000000e+02,  2.36289011e+00,
        7.16143375e+05,  9.09142841e+06,  1.29650511e+04,  2.14759190e+03,
        8.00000218e+03]) # all values in degrees or m

# a vector of elastic parameters
eparams = np.array([30e9, 30e9])  # 1st and 2nd Lame elastic parameters; try 30 GPa for both

In [6]:
model_los_disps_south = rect_shear_fault(fparams, eparams, data_s)
model_los_disps_north = rect_shear_fault(fparams, eparams, data_n)

In [12]:
#calculate the penalty from north and south 
print ('South Data Set')

penalty_s = np.sum(np.square(data_s[:,2]-model_los_disps_south))
print('total squared penalty:',penalty_s,'m^2')

RMS_s = np.sqrt(penalty_s/np.size(data_s[:,2]))
print('RMS:', RMS_s, 'm')

# calculate the mean residual
zero_shift_s = np.mean(data_s[:,2]-model_los_disps_south)
print('zero level shift:',zero_shift_s,'m')

# now subtract this from the data when computing the misfit
penalty_s = np.sum(np.square((data_s[:,2]-zero_shift_s)-model_los_disps_south))
print('total squared penalty:',penalty_s,'m^2')

South Data Set
total squared penalty: 0.7134331055748079 m^2
RMS: 0.07324041704231612 m
zero level shift: 0.028419122182327602 m
total squared penalty: 0.6060161203281373 m^2


In [13]:
#calculate the penalty from north and south 
print ('North Data Set')

penalty_n = np.sum(np.square(data_n[:,2]-model_los_disps_north))
print('total squared penalty:',penalty_n,'m^2')

RMS_n = np.sqrt(penalty_n/np.size(data_n[:,2]))
print('RMS:', RMS_n, 'm')

# calculate the mean residual
zero_shift_n = np.mean(data_n[:,2]-model_los_disps_north)
print('zero level shift:',zero_shift_n,'m')

# now subtract this from the data when computing the misfit
penalty_n = np.sum(np.square((data_n[:,2]-zero_shift_n)-model_los_disps_north))
print('total squared penalty:',penalty_s,'m^2')

North Data Set
total squared penalty: 0.05915942322277662 m^2
RMS: 0.05307650508050423 m
zero level shift: -0.02495629017933924 m
total squared penalty: 0.6060161203281373 m^2


In [14]:
#calculate the shift
zero_shift_n - zero_shift_s

-0.053375412361666846

In [15]:
data = np.vstack((data_s, data_n))

In [16]:
data

array([[ 7.12981608e+05,  9.09261116e+06,  0.00000000e+00, ...,
         1.07772000e-01, -8.55679000e-01,  1.00000000e+00],
       [ 7.12068114e+05,  9.09356676e+06,  0.00000000e+00, ...,
         1.08043000e-01, -8.54934000e-01,  2.00000000e+00],
       [ 7.11213952e+05,  9.09455177e+06,  0.00000000e+00, ...,
         1.08277000e-01, -8.54290000e-01,  3.00000000e+00],
       ...,
       [ 7.24117005e+05,  9.09558781e+06, -1.18000000e-01, ...,
         1.05098000e-01, -8.62781000e-01,  1.90000000e+01],
       [ 7.22902388e+05,  9.09570626e+06, -1.18000000e-01, ...,
         1.05405000e-01, -8.61972000e-01,  2.00000000e+01],
       [ 7.19525279e+05,  9.09503305e+06, -1.18000000e-01, ...,
         1.06223000e-01, -8.59810000e-01,  2.10000000e+01]])

In [17]:
model_los_disps = rect_shear_fault(fparams, eparams, data)

In [18]:
penalty = np.sum(np.square(data[:,2]-model_los_disps))
print('total squared penalty:',penalty,'m^2')

RMS = np.sqrt(penalty/np.size(data[:,2]))
print('RMS:', RMS, 'm')

total squared penalty: 0.7725925287975846 m^2
RMS: 0.07082961670503178 m


In [19]:
# calculate the mean residual
zero_shift = np.mean(data[:,2]-model_los_disps)
print('zero level shift:',zero_shift,'m')

# now subtract this from the data when computing the misfit
penalty = np.sum(np.square((data[:,2]-zero_shift)-model_los_disps))
print('total squared penalty:',penalty,'m^2')

zero level shift: 0.021140656860282124 m
total squared penalty: 0.7037657134350187 m^2


In [20]:
# i got this from tht digit NB
# this will save the file
np.savetxt('dsc_combine.okinv',data,fmt='%f %f %f %f %f %f %d')

In [22]:
# substract the shift from the 3 column of displacement
data_n_shifted = data_n

data_n_shifted [:,2] -= .118

In [23]:
# create a new stack of the data shifted
data_shifted = np.vstack((data_s, data_n_shifted))

In [24]:
#save again
np.savetxt('data_dsc_combined_shifted.okinv',data_shifted,fmt='%f %f %f %f %f %f %d')